In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações visuais para os gráficos
sns.set_theme(style="whitegrid")
np.random.seed(42) # Garante que os resultados aleatórios sejam os mesmos toda vez que rodar
tamanho_amostra = 100000 # Gerar muitos pontos para ter uma curva suave

# --- GRÁFICO 1: VARIANDO O PARÂMETRO μ (mu), O "CENTRO DE GRAVIDADE" ---

# Mantemos o sigma (espalhamento) fixo para ver apenas o efeito do mu
sigma_fixo = 0.5
mus_variaveis = [1.0, 2.0, 3.0]

# Cria uma figura com 3 gráficos lado a lado
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)
fig.suptitle('Efeito da Variação de μ (Centro de Gravidade) com σ Fixo', fontsize=16, y=1.02)

for i, mu in enumerate(mus_variaveis):
    # Gera 100.000 números aleatórios seguindo a distribuição Log-Normal com os parâmetros
    amostra = np.random.lognormal(mean=mu, sigma=sigma_fixo, size=tamanho_amostra)
    
    # Plota o histograma (barras) e a curva de densidade (a "rampa de skate")
    sns.histplot(amostra, bins=100, kde=True, ax=axes[i], line_kws={'linewidth': 3, 'color': 'red'})
    axes[i].set_title(f'μ = {mu:.1f} | σ = {sigma_fixo:.1f}')
    axes[i].set_xlabel('Valor Gerado')
    if i == 0:
        axes[i].set_ylabel('Frequência')

plt.show()


# --- GRÁFICO 2: VARIANDO O PARÂMETRO σ (sigma), O "FATOR ESPALHAMENTO" ---

# Mantemos o mu (centro) fixo para ver apenas o efeito do sigma
mu_fixo = 2.0
sigmas_variaveis = [0.2, 0.7, 1.5]

# Cria uma segunda figura com 3 gráficos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Efeito da Variação de σ (Fator Espalhamento) com μ Fixo', fontsize=16, y=1.02)

for i, sigma in enumerate(sigmas_variaveis):
    # Gera os dados
    amostra = np.random.lognormal(mean=mu_fixo, sigma=sigma, size=tamanho_amostra)
    
    # Plota o gráfico
    sns.histplot(amostra, bins=100, kde=True, ax=axes[i], line_kws={'linewidth': 3, 'color': 'red'})
    axes[i].set_title(f'μ = {mu_fixo:.1f} | σ = {sigma:.1f}')
    axes[i].set_xlabel('Valor Gerado')
    if i == 0:
        axes[i].set_ylabel('Frequência')

plt.show()

In [0]:
# Converte o resultado da sua query Spark para um DataFrame Pandas
df_comparativo_pd = df_comparativo.toPandas()
df_comparativo_pd = df_comparativo_pd.sort_values('mes')

# Inicia a criação do gráfico
fig, ax1 = plt.subplots(figsize=(14, 8))
plt.title('Comparativo de Tendência: Volume de Transações Real vs. Sintético', fontsize=16)

# --- Linha para os Dados REAIS (Eixo da Esquerda) ---
cor_real = 'tab:blue'
ax1.set_xlabel('Mês do Ano')
ax1.set_ylabel('Volume REAL de Transações', color=cor_real, fontsize=12)
ax1.plot(df_comparativo_pd['mes'], df_comparativo_pd['total_tx_pf_pagador_REAL'], color=cor_real, marker='o', label='Volume Real')
ax1.tick_params(axis='y', labelcolor=cor_real)

# --- Linha para os Dados SINTÉTICOS (Eixo da Direita) ---
# Cria um segundo eixo Y que compartilha o mesmo eixo X
ax2 = ax1.twinx()  
cor_sintetico = 'tab:orange'
ax2.set_ylabel('Volume SINTÉTICO de Transações', color=cor_sintetico, fontsize=12)
ax2.plot(df_comparativo_pd['mes'], df_comparativo_pd['total_tx_pf_pagador_SINTETICO'], color=cor_sintetico, marker='x', linestyle='--', label='Volume Sintético')
ax2.tick_params(axis='y', labelcolor=cor_sintetico)

# Adiciona a legenda
fig.legend(loc="upper right", bbox_to_anchor=(0.9, 0.9))
plt.grid(True)
plt.show()

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# --- 1. Geração de Dados de Exemplo ---
# (Você pode pular esta parte e usar seu DataFrame real)
# Nós simulamos o "Ataque de Madrugada" (p_madrugada=0.70)
# e transações legítimas com pico às 14h (loc=14).

print("Gerando dados de exemplo...")

def criar_dados_exemplo(n_legit=50000, n_fraude=5000, p_madrugada=0.7):
    # Gerar horas para transações legítimas (pico em horário comercial)
    horas_legitimas = np.random.normal(loc=14, scale=4.0, size=n_legit)
    # Garantir que as horas estão no intervalo [0, 23]
    horas_legitimas = np.clip(horas_legitimas, 0, 23).astype(int)
    
    # Gerar horas para transações fraudulentas
    n_madrugada = int(n_fraude * p_madrugada)
    n_fraude_normal = n_fraude - n_madrugada
    
    # Fraudes de "Ataque de Madrugada" (1h-4h)
    # np.random.randint(1, 5) gera números inteiros: 1, 2, 3, 4
    horas_fraude_madrugada = np.random.randint(1, 5, size=n_madrugada)
    
    # Outras fraudes que seguem o padrão normal
    horas_fraude_normal = np.random.normal(loc=14, scale=4.0, size=n_fraude_normal)
    horas_fraude_normal = np.clip(horas_fraude_normal, 0, 23).astype(int)
    
    horas_fraude = np.concatenate([horas_fraude_madrugada, horas_fraude_normal])
    
    # Criar DataFrames
    df_legit = pd.DataFrame({'hora': horas_legitimas, 'is_fraud': 0})
    df_fraude = pd.DataFrame({'hora': horas_fraude, 'is_fraud': 1})
    
    # Combinar e embaralhar
    df_transacoes = pd.concat([df_legit, df_fraude]).sample(frac=1).reset_index(drop=True)
    
    # Adicionar uma coluna 'data' apenas para ser fiel ao schema
    # (A hora é a única coisa que importa para este gráfico)
    base_date = datetime(2023, 1, 1)
    df_transacoes['data'] = df_transacoes['hora'].apply(
        lambda h: base_date + timedelta(hours=int(h), minutes=np.random.randint(0, 60))
    )
    
    return df_transacoes

# Gerar os dados de exemplo
# No seu caso, você carregaria seu DataFrame de transações aqui
df_transacoes_exemplo = criar_dados_exemplo()

print(df_transacoes_exemplo.head())


# --- 2. Código para Geração da Visualização ---
# (Assumindo que você tem um DataFrame chamado 'df_transacoes'
# com as colunas 'data' [datetime] e 'is_fraud' [0 ou 1])

# Para o exemplo, usaremos 'df_transacoes_exemplo'
df_para_plotar = df_transacoes_exemplo

# **Passo A: Extrair a hora da coluna 'data'**
# (Seu DataFrame 'df_transacoes' real pode já ter a coluna 'hora' 
# do passo de geração de dados, mas caso contrário, crie-a assim)
if 'hora' not in df_para_plotar.columns:
    df_para_plotar['hora'] = df_para_plotar['data'].dt.hour

# **Passo B: Plotar o gráfico**
print("Gerando visualização...")

# Configurar o estilo do gráfico
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 8))

# Criar o histograma com seaborn
# - 'x' é a coluna da hora
# - 'hue' é a coluna que separa as cores (fraude vs. legítima)
# - 'bins=24' cria uma barra para cada hora
# - 'multiple="stack"' empilha as barras de fraude sobre as legítimas
viz = sns.histplot(
    data=df_para_plotar,
    x='hora',
    hue='is_fraud',
    bins=24,
    multiple="stack",
    palette={0: "cornflowerblue", 1: "red"}, # 0=Legítima, 1=Fraude
    edgecolor="white",
    linewidth=0.5
)

# **Passo C: Melhorar os rótulos e a legenda**

# Mapear a legenda para nomes mais claros
try:
    handles = viz.legend_.legendHandles
    viz.legend_.remove()
    viz.legend(handles, ['Legítima', 'Fraude'], title='Tipo de Transação')
except AttributeError:
    # Caso a legenda não seja gerada automaticamente
    pass

# Configurar rótulos e título
plt.title('Distribuição de Transações por Hora do Dia (Legítimas vs. Fraudes)', fontsize=16, weight='bold')
plt.xlabel('Hora do Dia (0-23)', fontsize=12)
plt.ylabel('Contagem de Transações', fontsize=12)

# Força a exibição de todas as 24 horas no eixo X
plt.xticks(range(0, 24))
plt.xlim(-0.5, 23.5) # Ajusta os limites para centralizar as barras

# Exibir o gráfico
plt.show()